
# **Optimización de Memoria con Pandas**


Cuando trabajamos con grandes volúmenes de datos (cientos de miles o millones de filas), la biblioteca `pandas` puede consumir mucha memoria RAM si no se usan técnicas adecuadas.

Para ello deberemos:
- Verificar y optimizar el uso de memoria
- Usar tipos de datos más eficientes (`dtypes`)
- Cargar solo los datos necesarios
- Procesar archivos por bloques (chunks)
- Eliminar columnas innecesarias
- Convertir tipos de forma automática



## Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import sys

## Cargar un dataset grande

Usaremos un archivo CSV de ejemplo. Si no lo tienes, puedes crear uno pequeño para probar.

In [ ]:
big_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008.zip"
small_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008_small.zip"
very_small_csv = 'https://github.com/ricardoahumada/data-for-auditors/raw/refs/heads/main/4.%20An%C3%A1lisis%20Masivo%20de%20Datos/Optimizacion/data/2008_very_small.csv'

In [ ]:
df = pd.read_csv(small_csv)

In [ ]:
df.info()

## Verificar el uso de memoria

### Mostrar uso de memoria por columna



In [ ]:
df.memory_usage(deep=True)


### Total de memoria usada



In [ ]:
print(f"Memoria total usada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## Optimización básica – Tipos de datos (`dtypes`)

Pandas asigna automáticamente tipos de datos que pueden no ser óptimos. Podemos reducir el tamaño del DataFrame cambiando los tipos manualmente o automáticamente.

### Verificar tipos actuales



In [ ]:
df.dtypes

### Cambiar tipos manualmente (ejemplo)



In [ ]:
dtypes_optimized = {
    'Month': np.int8,
    'DepTime': np.float32,
    'UniqueCarrier': 'category'
}

df = df.astype(dtypes_optimized)



### Cambiar tipos automáticamente

Función para optimizar todos los tipos numéricos y categóricos:

In [ ]:
def optimize_dtypes(df):
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    for col in df.select_dtypes(include=['object']).columns:
        try:
            df[col] = df[col].astype('category')
        except:
            pass
    
    return df

df = optimize_dtypes(df)

### Verificar memoria después de optimizar

In [ ]:
print(f"Memoria después de optimizar: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## Eliminar columnas innecesarias

Si hay columnas que no se van a usar, elimínalas para liberar memoria.



In [ ]:
df.drop(['Unnamed: 0','FlightNum','CancellationCode'], axis=1, inplace=True)

# También puedes usar `del`:

# del df['CancellationCode']



## Cargar solo las columnas necesarias

En lugar de cargar todo el archivo, carga solo las columnas que necesitas.



In [ ]:
df.columns

In [ ]:
cols_to_use = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime',
       'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay']

df = pd.read_csv(small_csv, usecols=cols_to_use)



In [ ]:
print(f"Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## Cargar datos por partes (`chunksize`)

Si el archivo es muy grande para cargarlo en memoria, usa `chunksize`.

In [ ]:
def process_chunk(chunk):
    print(chunk.shape)
    print(chunk.head())

for chunk in pd.read_csv(small_csv, chunksize=100000):  # 100,000 filas por bloque
    process_chunk(chunk)  # función de procesamiento


## Limpiar valores nulos

Los valores nulos también ocupan espacio. Limpia o reemplaza según sea necesario.

In [ ]:
df.dropna(inplace=True)  # Elimina filas con nulos
# o
df.fillna(0, inplace=True)  # Rellena nulos con 0 u otros valores

In [ ]:
print(f"Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## Visualización opcional

Si quieres ver cómo afecta cada cambio al uso de memoria:


In [ ]:
before = sys.getsizeof(df)
df = optimize_dtypes(df)
after = sys.getsizeof(df)

print(f"Antes: {before / 1024**2:.2f} MB | Después: {after / 1024**2:.2f} MB")

## Más información:

- [Documentación oficial de pandas](https://pandas.pydata.org/pandas-docs/stable/)
- [Categorías en pandas](https://pandas.pydata.org/docs/user_guide/enhancingperf.html)
- [Uso de chunks](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
- [Downcasting numérico](https://pandas.pydata.org/docs/user_guide/enhancingperf.html#memory-usage)
